In [1]:
from dotenv import load_dotenv
import openai
import os

In [2]:
from openai import OpenAI

In [3]:
# Load environment variables from the .env file
load_dotenv()

True

In [4]:
# Access the variables
api_key = os.getenv('OPENAI_API_KEY')

In [5]:
client = openai.OpenAI(api_key=api_key)

In [6]:
response=client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role":"user","content":"is it too late to join the course?"}]
)

In [7]:
response

ChatCompletion(id='chatcmpl-9cknNDRaoMHhLhqf75ggx9jHsoG2t', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="It's hard to provide a definitive answer without more details, but generally speaking, whether it's too late to join a course can depend on several factors:\n\n1. **Enrollment Deadlines**: Most courses have specific enrollment periods. Check the course's official website or contact the institution to find out if enrollment is still open.\n   \n2. **Course Progress**: If the course has already started, you might want to consider how far along it is. Some courses may allow late joiners if the instructors feel they can catch up.\n   \n3. **Self-paced vs. Scheduled**: Many online courses are self-paced, which means you can start them at any time. Scheduled courses, however, may have a fixed timeline.\n   \n4. **Resources and Support**: If the course has already begun, consider whether you'll be able to access supplementary material

In [8]:
response.choices[0].message.content

"It's hard to provide a definitive answer without more details, but generally speaking, whether it's too late to join a course can depend on several factors:\n\n1. **Enrollment Deadlines**: Most courses have specific enrollment periods. Check the course's official website or contact the institution to find out if enrollment is still open.\n   \n2. **Course Progress**: If the course has already started, you might want to consider how far along it is. Some courses may allow late joiners if the instructors feel they can catch up.\n   \n3. **Self-paced vs. Scheduled**: Many online courses are self-paced, which means you can start them at any time. Scheduled courses, however, may have a fixed timeline.\n   \n4. **Resources and Support**: If the course has already begun, consider whether you'll be able to access supplementary materials such as recorded lectures, notes, and assignments. Additionally, check if there's support available, like office hours or forums.\n\n5. **Course Policy**: Eac

## Q1. Running Elastic 

Run Elastic Search 8.4.3, and get the cluster information. If you run it on localhost, this is how you do it:

```bash
curl localhost:9200
```

What's the `version.build_hash` value?

>Answer: 42f05b9372a9a4a470db3b52817899b99a76ee73

## Getting the data

Now let's get the FAQ data. You can run this snippet:

```python
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
```

Note that you need to have the `requests` library:

```bash
pip install requests
```

In [222]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the `course` field a keyword and the rest should be text. 

Don't forget to install the ElasticSearch client for Python:

```bash
pip install elasticsearch
```

Which function do you use for adding your data to elastic?

* `insert`
* `index`
* `put`
* `add`

> Answer: `index`

In [223]:
from elasticsearch import Elasticsearch

In [224]:
es_client=Elasticsearch('http://localhost:9200')

In [225]:
index_settings={
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name='course-questions'

es_client.indices.create(index=index_name,body=index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/2qMWkr2pSoCDy96m0GhMuw] already exists')

In [226]:
from tqdm.auto import tqdm

In [227]:
for doc in tqdm(documents):
    es_client.index(index=index_name,document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [228]:
query='i just discovered the course. Can i still join it'

In [229]:
def elastic_search(query):
    search_query={
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
    }
    response=es_client.search(index=index_name, body=search_query)
    
    result_docs=[]
    
    for response in response['hits']['hits']:
        result_docs.append(response['_source'])
    return result_docs

In [230]:
elastic_search(query)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minu

In [231]:
def build_prompt(query,search_results):
    prompt_template="""
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from CONTEXT when answering the QUESTION.

    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()
    
    context=""
    
    for doc in search_results:
        context=context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
        
    prompt=prompt_template.format(question=query,context=context).strip()
    return prompt
        


In [232]:
def llm(prompt):
    response=client.chat.completions.create(model='gpt-4o', messages=[{"role":"user","content":prompt}])
    return response.choices[0].message.content

In [233]:
def rag(query):
    search_results=elastic_search(query)
    prompt=build_prompt(query,search_results)
    answer=llm(prompt)
    return answer

In [234]:
rag(query)

'Yes, you can still join the course even if you discovered it after the start date. You are eligible to submit the homeworks regardless of your registration status. However, keep in mind that there will be deadlines for turning in the final projects, so try not to leave everything for the last minute.'

## Q3. Searching

Now let's search in our index. 

We will execute a query "How do I execute a command in a running docker container?". 

Use only `question` and `text` fields and give `question` a boost of 4, and use `"type": "best_fields"`.

What's the score for the top ranking result?

* 94.05
* 84.05
* 74.05
* 64.05

Look at the `_score` field.

> Answer: 84.05

In [235]:
query='How do I execute a command in a running docker container?'

In [236]:
def elastic_search(query):
    search_query={
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
    }
    response=es_client.search(index=index_name, body=search_query)

    top_score = response['hits']['hits'][0]['_score']
    
    # result_docs=[]
    
    # for response in response['hits']['hits']:
    #     result_docs.append(response['_source'])
        
    return top_score

In [237]:
elastic_search(query)

84.220634

## Q4. Filtering

Now let's only limit the questions to `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* How do I copy files from a different folder into docker container’s working directory?
* How do Lambda container images work?
* How can I annotate a graph?

In [238]:
query='How do I execute a command in a running docker container?'

In [239]:
def elastic_search(query):
    search_query={
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
    }
    response=es_client.search(index=index_name, body=search_query)

    # top_score = response['hits']['hits'][0]['_score']
    
    result_docs=[]
    
    for response in response['hits']['hits']:
        result_docs.append(response['_source'])
        
    return result_docs

In [240]:
search_results=elastic_search(query)

In [241]:
search_results

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Launch the container im

In [242]:
# Retrieve the 3rd question
if len(search_results) >= 3:
    third_question = search_results[2]['question']
else:
    third_question = None

third_question

'How do I debug a docker container?'

In [243]:
def rag(query):
    search_results=elastic_search(query)
    prompt=build_prompt(query,search_results)
    answer=llm(prompt)
    return answer

In [244]:
results=rag(third_question)

In [220]:
results

'To execute a command in a running Docker container, follow these steps:\n\n1. First, find the container ID of the running container by using the command:\n   ```\n   docker ps\n   ```\n   \n2. Once you have the container ID, use the following command to execute a command inside the container:\n   ```\n   docker exec -it <container-id> <command>\n   ```\n\nFor example, to start a bash session inside the container:\n```\ndocker exec -it <container-id> bash\n```\n\nThis will allow you to interact with the running container.'

## Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM. 

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (`\n\n`)
```python
context_template = """
Q: {question}
A: {text}
""".strip()
```

Now use the context you just created along with the "How do I execute a command in a running docker container?" question 
to construct a prompt using the template below:

```
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

What's the length of the resulting prompt? (use the `len` function)

* 962
* 1462
* 1962
* 2462

In [245]:
prompt = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani
"""

length_of_prompt = len(prompt)
length_of_prompt

653

## Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of 
tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

```bash
pip install tiktoken
```

Let's calculate the number of tokens in our query: 

```python
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the `encode` function. How many tokens does our prompt have?

* 122
* 222
* 322
* 422

Note: to decode back a token into a word, you can use the `decode_single_token_bytes` function:

```python
encoding.decode_single_token_bytes(63842)
```

In [246]:
import tiktoken

# The prompt we constructed
prompt = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani
"""

# Get the encoding for the "gpt-4o" model
encoding = tiktoken.encoding_for_model("gpt-4o")

# Encode the prompt
tokens = encoding.encode(prompt)

# Number of tokens
num_tokens = len(tokens)
num_tokens


141